In [78]:
from fastkml import kml, styles
# import lxml
from shapely.geometry import Point, LineString, Polygon
import re
import requests
import folium
from shapely import wkt, geometry
import json
import copy
from pprint import pprint, pformat
from heapq import nsmallest
import csv

# Einlesen der Daten von einem KML-File

In [79]:
read_kml = False
delta_time = 5

In [80]:
if read_kml:
    print("read kml")
    with open("Rettungstreffpunkte_BaySF copy.kml", 'rt', encoding="utf-8") as myfile:
        doc=myfile.read()

    k = kml.KML()
    k.from_string(doc)
    features = list(k.features())
    print("len features: "+str(len(features)))
    f2 = list(features[0].features())

    print("len f2: "+str(len(f2)))
    print("f2[0].name: "+str(f2[0].name))
    rps = list(f2[0].features())
    rps_names=[]
    for rp in rps:
        rps_names.append(rp.name)
    rps_filtered=[]
    for rp in rps:
        if re.match("((PA\-(S|L))|(FRG))\-[0-9]+",rp.name):
            rps_filtered.append(rp)

    print("rps[0].extended_data.elements[0].data[2]: "+str(rps[0].extended_data.elements[0].data[2]))        
    print("rps[0].extended_data.elements[0].data[3]: "+str(rps[0].extended_data.elements[0].data[3]))      

    def get_coords():
        coords=[]
        for rp in rps_filtered:
            coords.append({"name":rp.name,"lon":rp.extended_data.elements[0].data[3]["value"],"lat":rp.extended_data.elements[0].data[2]["value"]})
        return coords

    coords = get_coords()
    print("coords[1]:"+str(coords[1]))
else:
    print("skip kml")

skip kml


In [81]:

if not read_kml:
    print("read csv")
    def read_coords_from_csv_file(filename:str)->list:
        coords =[]
        with open(filename,encoding="utf-8") as csvdatei:
            csv_reader_object = csv.reader(csvdatei, delimiter=',')
            row_header = True
            for row in csv_reader_object:
                if row_header:
                    row_header = False
                    index_lat = row.index("lat")
                    index_lon = row.index("lon")
                    index_name = row.index("ORTSTEIL")
                    
                    print("index_lat: "+str(index_lat))
                    print("index_lon: "+str(index_lon))
                    continue
                # print(row)
                if row[index_lon]=="" or row[index_lat]=="":
                    # print("skipped row because of missing coordinates")
                    pass
                else:
                    coords.append({"name":row[index_name],"lon":row[index_lon],"lat":row[index_lat]})
        return coords
    def get_coords():
        coords =[]
        coords.extend(read_coords_from_csv_file('OrteFRG_2.csv'))
        print(len(coords))
        coords.extend(read_coords_from_csv_file('OrtePA-L2.csv'))
        print(len(coords))
        return coords

read csv


In [82]:

def get_intervalls(length:int,chunksize:int):
    r= range(0, length,chunksize)
    intervalls=[]
    for i in r:
        intervalls.append((i,i+chunksize if  i+chunksize<length else length-1))
    return intervalls


In [83]:
def get_segments(input:list,chunksize:int):
    segments=[]
    for i in get_intervalls(len(input),chunksize):
        segments.append(input[i[0]:i[1]].copy())
    return segments
segments=get_segments(get_coords(),200)
print(len(segments))
# print(segments[5])
segments[0][0]

index_lat: 4
index_lon: 5
645
index_lat: 4
index_lon: 5
2352
12


{'name': 'Daxberg - Eppenschlag', 'lon': ' 13.314049', 'lat': '48.907756'}

In [84]:
standorte_alle =[
    {"color":"red","bereitschaft":"Passau Haidmühle","name":"HM 81/1", "lon": '13.731204',"lat": '48.767037', "offset":0, "cartype":"RFZ"}, #HM 81/1 (Altreichenau)
    # # {"color":"red","name":"HM 81/1", "lon": '13.753724',"lat": '48.747569', "offset":0, "cartype":"RFZ"}, #HM 81/1 (Neureichenau)
    {"color":"darkred","bereitschaft":"Passau Haidmühle","name":"PA 84/1", "lon": '13.436274',"lat": '48.56527', "offset":0, "cartype":"MTW"}, # PA 84/1 (Passau)
    {"color":"orange","bereitschaft":"Hauzenberg Waldkirchen","name":"WK 81/1", "lon": '13.658887',"lat": '48.68727', "offset":0, "cartype":"RFZ"}, # WK 81/1 (Hauzenberg)
    {"color":"orange","bereitschaft":"Hauzenberg Waldkirchen","name":"WK 84/1", "lon": '13.63113',"lat": '48.65669', "offset":0, "cartype":"MTW"}, # WK 84/1 (Oberfrauenwald)
    {"color":"blue","bereitschaft":"Wolfstein","name":"Fi 81/1", "lon": '13.574553',"lat": '48.940853', "offset":0, "cartype":"RFZ"}, # Fi 81/1 (Finsterau)
    {"color":"cadetblue","bereitschaft":"Wolfstein","name":"Mifi 84/1", "lon": '13.648276',"lat": '48.892684', "offset":0, "cartype":"MTW"}, # MiFi 84/1 (Mitterfirmiansreut)
    {"color":"cadetblue","bereitschaft":"Wolfstein","name":"Mifi 81/1 W", "lon": '13.648276',"lat": '48.892684', "offset":0, "cartype":"MTW"}, # MiFi 84/1 (Mitterfirmiansreut)
    # {"color":"lightblue","bereitschaft":"Wolfstein","name":"Mifi 81/1 S", "lon": '13.548798',"lat": '48.805645', "offset":0, "cartype":"RFZ"}, # MiFi 81/1 (Freyung)
    {"color":"purple","bereitschaft":"Schöfweg","name":"Schöf 81/1", "lon": '13.230549',"lat": '48.837513', "offset":0, "cartype":"RFZ"}, # Schöf 81/1 (Schöfweg)
    {"color":"beige","bereitschaft":"Grafenau","name":"Gr 81/1", "lon": '13.396411',"lat": '48.86228', "offset":0, "cartype":"RFZ"},# Gr 81/1 (Grafenau)
    # {"color": "beige","bereitschaft":"Deggendorf", "name": "Deg 81/1", "lon": "12.956874683480482", "lat": "48.833301710656386", "offset": 0,
        # "cartype": "RFZ"}  # Deg 81/1 (Deggendorf)
]

# cadetblue', 'red', 'lightblue', 'gray', 'black', 'orange', 'darkgreen', 'lightred', 'darkred', 'beige', 'purple', 'darkpurple', 'pink', 'darkblue', 'blue', 'lightgray', 'white', 'lightgreen', 'green'}.
#   icon = folium.map.Icon(color=rwachen[dur.index(min(dur))]["color"] ,
standorte_81=[x for x in standorte_alle if x["cartype"] == "RFZ" ]
standorte_84=[x for x in standorte_alle if x["cartype"] == "MTW" ]



In [85]:
headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': '5b3ce3597851110001cf6248d7c919c8cfba4867bca79756059b142a',
    'Content-Type': 'application/json; charset=utf-8'
}

In [86]:
def min_with_offset_index(mylist) -> int:
    with_offset = [x+standorte_alle[i]["offset"]*60 for i,x in enumerate(mylist)] 
    return with_offset.index(min(with_offset)) 
def min_with_offset_value(mylist) -> float:
    with_offset = [x+standorte_alle[i]["offset"]*60 for i,x in enumerate(mylist)] 
    return min(with_offset)
def min_with_offset_list(mylist) -> list:
    with_offset = [x+standorte_alle[i]["offset"]*60 for i,x in enumerate(mylist)] 
    return with_offset

In [87]:
result=[]
segments = get_segments(get_coords(),200)
for i, seg in enumerate(segments):
    seg_result = copy.deepcopy(seg)
    body={}
    body["locations"]=[]
    for w in standorte_alle:
        body["locations"].append([w["lon"],w["lat"]])
    for x in seg:
        body["locations"].append([x["lon"][1:-1],x["lat"][:-1]])
    body["metrics"]=["distance","duration"]
    body["destinations"]=list(range(0,len(standorte_alle)))
    body["sources"]=list(range(len(standorte_alle),len(standorte_alle)+len(seg)))
    body["units"]="km"
    # print("========== new body =============\n"*10)

    call = requests.post('https://api.openrouteservice.org/v2/matrix/driving-car', json=body, headers=headers)
    print(call.status_code, call.reason, "Segement:", i+1,"/",len(segments) )
    if call.status_code != 200:
        print(body)
        print(call.json())
    
    fetched_data=call.json()
    
    for i, p in enumerate(seg_result):
        p["durations"]=fetched_data["durations"][i]
        p["distances"]=fetched_data["distances"][i]
    
    result.extend(seg_result)
result_tmp = copy.deepcopy(result)
len(result)

index_lat: 4
index_lon: 5
645
index_lat: 4
index_lon: 5
2352
200 OK Segement: 1 / 12
200 OK Segement: 2 / 12
200 OK Segement: 3 / 12
200 OK Segement: 4 / 12
200 OK Segement: 5 / 12
200 OK Segement: 6 / 12
200 OK Segement: 7 / 12
200 OK Segement: 8 / 12
200 OK Segement: 9 / 12
200 OK Segement: 10 / 12
200 OK Segement: 11 / 12
200 OK Segement: 12 / 12


2351

In [88]:
segments[0][0]

{'name': 'Daxberg - Eppenschlag', 'lon': ' 13.314049', 'lat': '48.907756'}

In [89]:
result = copy.deepcopy(result_tmp)
for i, p in enumerate(result):
    p["offset_durations"] = min_with_offset_list(p["durations"])
    p["durations81er"] = [x for i, x in enumerate(p["durations"]) if standorte_alle[i]["cartype"] == "RFZ"]
    p["durations84er"] = [x for i, x in enumerate(p["durations"]) if standorte_alle[i]["cartype"] == "MTW"]

    p["offset_durations81er"] = [x for i, x in enumerate(p["offset_durations"]) if standorte_alle[i]["cartype"] == "RFZ"]
    p["offset_durations84er"] = [x for i, x in enumerate(p["offset_durations"]) if standorte_alle[i]["cartype"] == "MTW"]
    
    p["distances81er"] = [x for i, x in enumerate(p["distances"]) if standorte_alle[i]["cartype"] == "RFZ"]
    p["distances84er"] = [x for i, x in enumerate(p["distances"]) if standorte_alle[i]["cartype"] == "MTW"]




    p["index_fastest"] = p["durations"].index(min(p["durations"]))
    p["index_fastest81er"] = p["durations81er"].index(min(p["durations81er"]))
    p["index_fastest84er"] = p["durations84er"].index(min(p["durations84er"]))

    p["index_fastest_offset"] = min_with_offset_index(p["durations"])
    p["index_fastest_offset81er"] = min_with_offset_index(p["offset_durations81er"])
    p["index_fastest_offset84er"] = min_with_offset_index(p["offset_durations84er"])

    p["indexes_fastest"] = [p["durations"].index(x)for x in nsmallest(len(standorte_alle), p["durations"])]
    p["indexes_fastest81er"] = [p["durations81er"].index(x)for x in nsmallest(len(standorte_alle), p["durations81er"])]
    p["indexes_fastest84er"] = [p["durations84er"].index(x)for x in nsmallest(len(standorte_alle), p["durations84er"])]
    p["indexes_fastest_offset"] = [p["offset_durations"].index(x)for x in nsmallest(len(standorte_alle), p["offset_durations"])]
    p["indexes_fastest_offset81er"] = [p["offset_durations81er"].index(x)for x in nsmallest(len(standorte_alle), p["offset_durations81er"])]
    p["indexes_fastest_offset84er"] = [p["offset_durations84er"].index(x)for x in nsmallest(len(standorte_alle), p["offset_durations84er"])]

    p["index_shortest"] = p["distances"].index(min(p["distances"]))
    p["index_shortest81er"] = p["distances81er"].index(min(p["distances81er"]))
    p["index_shortest84er"] = p["distances84er"].index(min(p["distances84er"]))

    p["fastest_time"] = "{0:d}:{1:02d} min".format(int(min(p["durations"])//60), int(min(p["durations"]) % 60))
    p["fastest_time_offset"] = "{0:d}:{1:02d} min".format(int(min(p["offset_durations"])//60), int(min(p["offset_durations"]) % 60))
    p["fastest_time81er"] = "{0:d}:{1:02d} min".format(int(min(p["durations81er"])//60), int(min(p["durations81er"]) % 60))
    p["fastest_time_offset81er"] = "{0:d}:{1:02d} min".format(int(min(p["offset_durations81er"])//60), int(min(p["offset_durations81er"]) % 60))
    p["fastest_time84er"] = "{0:d}:{1:02d} min".format(int(min(p["durations84er"])//60), int(min(p["durations84er"]) % 60))
    p["fastest_time_offset84er"] = "{0:d}:{1:02d} min".format(int(min(p["offset_durations84er"])//60), int(min(p["offset_durations84er"]) % 60))

    p["shortest_dist"] = "{0:.1f} km".format(min(p["distances"]))
    p["shortest_dist81er"] = "{0:.1f} km".format(min(p["distances81er"]))
    p["shortest_dist84er"] = "{0:.1f} km".format(min(p["distances84er"]))
  
    p["index_in"+str(delta_time)+"min"] = [p["durations"].index(x) for x in p["durations"] if (x-min(p["durations"]) < delta_time*60)]
    p["index_in"+str(delta_time)+"min81er"] = [p["durations81er"].index(x) for x in p["durations81er"] if (x-min(p["durations81er"]) < delta_time*60)]
    p["index_in"+str(delta_time)+"min84er"] = [p["durations84er"].index(x) for x in p["durations84er"] if (x-min(p["durations84er"]) < delta_time*60)]

    p["formated_durations"] = ["{0:d}:{1:02d} min".format(int(x//60), int(x % 60)) for x in p["durations"]]
    p["formated_durations_offset"] = ["{0:d}:{1:02d} min".format(int(x//60), int(x % 60)) for x in p["offset_durations"]]
    p["formated_durations81er"] = ["{0:d}:{1:02d} min".format(int(x//60), int(x % 60)) for x in p["durations81er"]]
    p["formated_durations84er"] = ["{0:d}:{1:02d} min".format(int(x//60), int(x % 60)) for x in p["durations84er"]]
    p["formated_durations_offset81er"] = ["{1:0d}:{0:02d} min".format(int(x//60), int(x % 60)) for x in p["offset_durations81er"]]

    p["formated_distances"] = ["{0:.1f} km".format(x) for x in p["distances"]]
    p["formated_distances81er"] = ["{0:.1f} km".format(x) for x in p["distances81er"]]
    p["formated_distances84er"] = ["{0:.1f} km".format(x) for x in p["distances84er"]]

    p["names_in"+str(delta_time)+"min"] = [standorte_alle[p["durations"].index(
        x)]["name"] for x in p["durations"] if (x-min(p["durations"]) < delta_time*60)]
    p["names_in"+str(delta_time)+"min81er"] = [standorte_81[p["durations81er"].index(
        x)]["name"] for x in p["durations81er"] if (x-min(p["durations81er"]) < delta_time*60)]
    p["names_in"+str(delta_time)+"min84er"] = [standorte_81[p["durations84er"].index(
        x)]["name"] for x in p["durations84er"] if (x-min(p["durations84er"]) < delta_time*60)]

    p["2names_in"+str(delta_time)+"min"] = [standorte_alle[x]["name"]
                                       for x in p["index_in"+str(delta_time)+"min"]]
    p["2names_in"+str(delta_time)+"min81er"] = [standorte_81[x]["name"]
                                           for x in p["index_in"+str(delta_time)+"min81er"]]
    p["2names_in"+str(delta_time)+"min84er"] = [standorte_84[x]["name"]
                                           for x in p["index_in"+str(delta_time)+"min84er"]]                                           

    # p["color_fastest"]=standorte_alle[index_fastest]["color"]
    # p["color_shortest"]=standorte_alle[index_shortest]["color"]
result[0]


{'name': 'Daxberg - Eppenschlag',
 'lon': ' 13.314049',
 'lat': '48.907756',
 'durations': [3509.25,
  3454.61,
  3759.01,
  3601.95,
  2660.72,
  2593.0,
  2593.0,
  1137.09,
  1088.41],
 'distances': [48.84, 51.21, 53.08, 55.28, 34.12, 35.55, 35.55, 19.24, 13.48],
 'offset_durations': [3509.25,
  3454.61,
  3759.01,
  3601.95,
  2660.72,
  2593.0,
  2593.0,
  1137.09,
  1088.41],
 'durations81er': [3509.25, 3759.01, 2660.72, 1137.09, 1088.41],
 'durations84er': [3454.61, 3601.95, 2593.0, 2593.0],
 'offset_durations81er': [3509.25, 3759.01, 2660.72, 1137.09, 1088.41],
 'offset_durations84er': [3454.61, 3601.95, 2593.0, 2593.0],
 'distances81er': [48.84, 53.08, 34.12, 19.24, 13.48],
 'distances84er': [51.21, 55.28, 35.55, 35.55],
 'index_fastest': 8,
 'index_fastest81er': 4,
 'index_fastest84er': 2,
 'index_fastest_offset': 8,
 'index_fastest_offset81er': 4,
 'index_fastest_offset84er': 2,
 'indexes_fastest': [8, 7, 5, 5, 4, 1, 0, 3, 2],
 'indexes_fastest81er': [4, 3, 2, 0, 1],
 'index

In [90]:
[result[0]["durations"].index(x)for x in nsmallest(len(standorte_alle),result[0]["durations"])]

[8, 7, 5, 5, 4, 1, 0, 3, 2]

In [91]:
i=0
[x for x in result[i]["durations"] if standorte_alle[result[i]["durations"].index(x)]["cartype"] == "RFZ"]

[3509.25, 3759.01, 2660.72, 1137.09, 1088.41]

In [92]:
i=0
[result[i]["durations"].index(x) for x in result[i]["durations"] if (x-min(result[i]["durations"])<120)]

[7, 8]

In [93]:
nsmallest(2,result[0]["durations"])[1]-nsmallest(2,result[0]["durations"])[0]

48.679999999999836

In [94]:
result[0]

{'name': 'Daxberg - Eppenschlag',
 'lon': ' 13.314049',
 'lat': '48.907756',
 'durations': [3509.25,
  3454.61,
  3759.01,
  3601.95,
  2660.72,
  2593.0,
  2593.0,
  1137.09,
  1088.41],
 'distances': [48.84, 51.21, 53.08, 55.28, 34.12, 35.55, 35.55, 19.24, 13.48],
 'offset_durations': [3509.25,
  3454.61,
  3759.01,
  3601.95,
  2660.72,
  2593.0,
  2593.0,
  1137.09,
  1088.41],
 'durations81er': [3509.25, 3759.01, 2660.72, 1137.09, 1088.41],
 'durations84er': [3454.61, 3601.95, 2593.0, 2593.0],
 'offset_durations81er': [3509.25, 3759.01, 2660.72, 1137.09, 1088.41],
 'offset_durations84er': [3454.61, 3601.95, 2593.0, 2593.0],
 'distances81er': [48.84, 53.08, 34.12, 19.24, 13.48],
 'distances84er': [51.21, 55.28, 35.55, 35.55],
 'index_fastest': 8,
 'index_fastest81er': 4,
 'index_fastest84er': 2,
 'index_fastest_offset': 8,
 'index_fastest_offset81er': 4,
 'index_fastest_offset84er': 2,
 'indexes_fastest': [8, 7, 5, 5, 4, 1, 0, 3, 2],
 'indexes_fastest81er': [4, 3, 2, 0, 1],
 'index

In [95]:
len(result)

2351

In [96]:
api_key = 'your_key'
wkt_str = 'Point (13.65886 48.68729)'
aoi_geom = wkt.loads(wkt_str) # load geometry from WKT string

aoi_coords = list(aoi_geom.coords) # get coords from exterior ring
aoi_coords = [(y,x) for x,y in aoi_coords] # swap (x,y) to (y,x). Really leaflet?!
aoi_centroid = aoi_geom.centroid # Kreuzberg center for map center

In [97]:
# m = folium.Map(location=(aoi_centroid.y, aoi_centroid.x), zoom_start=11)
# # m = folium.Map(tiles='Stamen Toner',location=(aoi_centroid.y, aoi_centroid.x), zoom_start=10)
# points =[]
# for point in standorte_alle:
#     name = point["name"]
#     lon = point["lon"]
#     lat = point["lat"]
#     lon = float(lon)
#     lat = float(lat)
#     # popup = "<strong>{0}</strong0><tabel><br>Lat: {1}<br>Long: {2}".format(name, lat, lon)
#     # popup = "<table width='150px'><tr><td colspan='2'><strong>{0}</strong></td></tr><tr><td>Lat</td><td>{1}</td></tr><tr><td>Long</td><td>{2}</td></tr></table>".format(name, lat, lon)
#     tooltip = "<table width='150px'><tr><td colspan='2'><strong>{0}</strong></td></tr><tr><td>Lat</td><td>{1}</td></tr><tr><td>Long</td><td>{2}</td></tr></table>".format(name, lat, lon)
#     icon = folium.map.Icon(color='green',
#                         icon_color='#0000ff',
#                         icon="ambulance",
#                         prefix='fa')
#     # folium.map.Marker([lat, lon], icon=icon, popup=popup, tooltip=tooltip).add_to(m)
#     folium.map.Marker([lat, lon], icon=icon, tooltip=tooltip).add_to(m)
#     points.append(name)
# for point in result:
#     # name,lon,lat,dur,dist,fastest,shortest,ft,sd, cs, cf = point.values()
#     lon = float(point["lon"][:-1])
#     lat = float(point["lat"][:-1])
#     fastest = standorte_alle[point["index_fastest"]]["name"]
#     shortest = standorte_alle[point["index_shortest"]]["name"]
#     color_fastest = standorte_alle[point["index_fastest"]]["color"]
#     color_shortest = standorte_alle[point["index_shortest"]]["color"]
#     popup = "<strong>{0}</strong><br>Lat: {1}<br>Long: {2}<br>Fastest: {3}<br>Shortest: {4}".format(point["name"], point["lat"], point["lon"], fastest,shortest)
#     tooltip = "<table width='200px'><tr><td colspan='3'><strong>{0}</strong></td></tr><tr><td><i class='far fa-clock'></i></td><td>{1}</td><td>{2}</td></tr><tr><td><i class='fas fa-arrows-alt-h'></i></td><td>{3}</td><td>{4}</td></tr></table>".format(point["name"],fastest ,point["fastest_time"], shortest ,point["shortest_dist"])
#     # tooltip = "<strong>{0}</strong><brFastest: {1}<br>Shortest: {2}".format(name,fastest,shortest)
#     icon = folium.map.Icon(color=color_fastest ,
#                         icon_color=color_shortest,
#                         prefix='fa',
#                         icon="arrows")
#     folium.map.Marker([lat, lon], icon=icon, popup=popup, tooltip=tooltip).add_to(m)
#     points.append(name)

# m

In [98]:
# Create the root KML object
k = kml.KML()
ns = '{http://www.opengis.net/kml/2.2}'
# Create a KML Document and add it to the KML root object
d = kml.Document(ns, 'docid', 'Rettungspunkte Bergwachtbereitschaften ILS-Passau', 'Alle Rettungstreffpunkte mit Angaben zur Entfernung und Fahrzeit zum nächsten Bergwachtauto')
k.append(d)

# Create a KML Folder and add it to the Document
# fs = kml.Folder(ns, '10', 'Fahrzeugstandorte', 'Standorte an denen eine Bergwachtfahrzeug steht')
# d.append(fs)

# Create a KML Folder and nest it in the first Folder
fr = kml.Folder(ns, 'rt', 'Rettungstreffpunkte', 'Forst Rettungstreffpunkte')
d.append(fr)
fr2 =  kml.Folder(ns, 'rt2', 'Rettungstreffpunkte2', 'Forst Rettungstreffpunkte')
d.append(fr2)

# Create a Placemark with a simple polygon geometry and add it to the
# second folder of the Document
for i,point in enumerate(result[:5]):
    fastest = standorte_alle[point["index_fastest"]]["name"]
    shortest = standorte_alle[point["index_shortest"]]["name"]
    p = kml.Placemark(ns, 'klm_rp{0:04d}'.format(i), point['name'], "Fastest: {0} ({1})  Nearest: {2} ({3})".format(fastest, point["fastest_time"], shortest ,point["shortest_dist"]))
    p.geometry = Point(float(point["lon"][1:-1]),float(point["lat"][:-1]))
    #Style
    pms = kml.Style()
    # style = styles.BalloonStyle(ns=ns,id='klm_ico{0:04d}'.format(i),bgColor="#ff0000")
    style = styles.IconStyle(ns=ns,id='klm_ico{0:04d}'.format(i),color="7fff0000", scale=0.5)
    pms.append_style(style)
    p.append_style(pms)
    fr.append(p)
for i,point in enumerate(result[5:15]):
    fastest = standorte_alle[point["index_fastest"]]["name"]
    shortest = standorte_alle[point["index_shortest"]]["name"]
    p = kml.Placemark(ns, 'klm_rp{0:04d}'.format(i), point['name'], "Fastest: {0} ({1})  Nearest: {2} ({3})".format(fastest, point["fastest_time"], shortest ,point["shortest_dist"]))
    p.geometry = Point(float(point["lon"][1:-1]),float(point["lat"][:-1]))
    #Style
    pms = kml.Style()
    # style = styles.BalloonStyle(ns=ns,id='klm_ico{0:04d}'.format(i),bgColor="#ff0000")
    style = styles.IconStyle(ns=ns,id='klm_ico{0:04d}'.format(i),color="7fff0000", scale=0.5)
    pms.append_style(style)
    p.append_style(pms)
    fr2.append(p)

with open("Rettungstreffpunkte_Bergwacht.kml", 'w', encoding="utf-8") as myfile:
    myfile.write('<?xml version="1.0" encoding="UTF-8"?>')
    myfile.write(k.to_string(prettyprint=True))
print("fin")

fin


# Alle

In [99]:
result[0]["names_in"+str(delta_time)+"min"].sort()
result[0]["index_in"+str(delta_time)+"min"]


[7, 8]

In [100]:
titlerow = ["Rettungspunkt", "Longitude", "Latitude", "Koordinaten", "Schnellestes Fahrzeug",
            "Anfahrtszeit", "Nächstes Fahrzeug", "Anfahrtsdistanz", "Ankunft innerhalb "+str(delta_time)+"min"]
all_times_title = []
all_dist_title = []
o81er_times_title = []
o81er_dist_title = []

for w in standorte_alle:
    all_times_title.append("Zeit_"+w["name"])
    all_dist_title.append("Entf_"+w["name"])
    if (w in standorte_81):
        o81er_times_title.append("Zeit_"+w["name"])
        o81er_dist_title.append("Entf_"+w["name"])
all_titlerow = copy.deepcopy(titlerow)
all_titlerow.extend(all_times_title)
all_titlerow.extend(all_dist_title)

result_segments = get_segments(result,2000)
for i,segment in enumerate(result_segments):

    with open("Orte_Bergwacht_"+str(i)+".csv", 'w', encoding="utf-8",newline='') as myfile:
        writer = csv.writer(myfile,delimiter=",",quotechar='"', quoting=csv.QUOTE_MINIMAL)
        writer.writerow(all_titlerow)

        for p in segment:
            p["names_in"+str(delta_time)+"min"].sort()
            writer.writerow(
                [p["name"]
                ,p["lon"]
                ,p["lat"]
                ,p["lon"]+","+p["lat"]
                ,standorte_alle[p["index_fastest"]]["name"]
                ,p["fastest_time"]
                ,standorte_alle[p["index_shortest"]]["name"]
                ,p["shortest_dist"]
                 ,  ", ".join(p["names_in"+str(delta_time)+"min"])]
                +p["formated_durations"]
                +p["formated_distances"])
    print("fin_full_"+str(i))
    # exit()


    # nur 81er  
    with open("Orte_Bergwacht_nur81er_"+str(i)+".csv", 'w', encoding="utf-8",newline='') as myfile:
        writer = csv.writer(myfile,delimiter=",",quotechar='"', quoting=csv.QUOTE_MINIMAL)
        writer.writerow(titlerow)

        for p in segment:
            p["names_in"+str(delta_time)+"min81er"].sort()
            writer.writerow(
                [p["name"]
                ,p["lon"]
                ,p["lat"]
                ,p["lon"]+","+p["lat"]
                ,standorte_81[p["index_fastest81er"]]["name"]
                ,p["fastest_time81er"]
                ,standorte_81[p["index_shortest81er"]]["name"]
                ,p["shortest_dist81er"]
                 , ", ".join(p["names_in"+str(delta_time)+"min81er"])]
                +p["formated_durations81er"]
                +p["formated_distances81er"])
            # myfile.write("{0};{1};{2};{2},{1};{3};{4};{5};{6};{7};{8};{9};\n".format(p["name"],p["lon"],p["lat"],standorte_81[p["index_fastest81er"]]["name"],p["fastest_time81er"],standorte_81[p["index_shortest81er"]]["name"],p["shortest_dist81er"],in2min,time,dist))

    print("fin_81er_"+str(i))

    # Überschneidung 2min
    with open("Orte_Bergwacht_Überschneidungen_"+str(i)+".csv", 'w', encoding="utf-8",newline='') as myfile:
        # myfile.write(string+time_str+dist_str+'\n')
        # for p in [x for x in result if len(x["index_in"+str(delta_time)+"min"])>1]:
        #     p["names_in"+str(delta_time)+"min"].sort()
        #     in"+str(delta_time)+"min = ", ".join(p["names_in"+str(delta_time)+"min"])
        #     time = "; ".join(p["formated_durations"])
        #     dist = "; ".join(p["formated_distances"])
        #     myfile.write("{0};{1};{2};{2},{1};{3};{4};{5};{6};{7};{8};{9};\n".format(p["name"],p["lon"],p["lat"],standorte_alle[p["index_fastest"]]["name"],p["fastest_time"],standorte_alle[p["index_shortest"]]["name"],p["shortest_dist"],in2min,time,dist))

        writer = csv.writer(myfile,delimiter=",",quotechar='"', quoting=csv.QUOTE_MINIMAL)
        writer.writerow(all_titlerow)

        for p in [x for x in segment if len(x["index_in"+str(delta_time)+"min"]) > 1]:
            p["names_in"+str(delta_time)+"min"].sort()
            writer.writerow(
                [p["name"]
                ,p["lon"]
                ,p["lat"]
                ,p["lon"]+","+p["lat"]
                ,standorte_alle[p["index_fastest"]]["name"]
                ,p["fastest_time"]
                ,standorte_alle[p["index_shortest"]]["name"]
                ,p["shortest_dist"]
                 , ", ".join(p["names_in"+str(delta_time)+"min"])]
                +p["formated_durations"]
                +p["formated_distances"])
    print("fin_full_overlap_"+str(i))



fin_full_0
fin_81er_0
fin_full_overlap_0
fin_full_1
fin_81er_1
fin_full_overlap_1


# Gesamtdateien

In [104]:
titlerow2 = ["Rettungspunkt",
"Schnellestes Fahrzeug","Schn. Bereitschaft","Standort","Anfahrtszeit",
"schnellster 81er","Schn. Bereitschaft 81er","Standort 81er","Anfahrtszeit 81er",
"schnellster 84er","Schn. Bereitschaft 84er","Standort 84er","Anfahrtszeit 84er",
 "Ankunft innerhalb "+str(delta_time)+"min"]
# nur beide  
with open("Orte_Bergwacht_mix_gesamt.csv", 'w', encoding="utf-8", newline='') as myfile:
    writer = csv.writer(myfile,delimiter=",",quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(titlerow2)

    for p in result:
        p["names_in"+str(delta_time)+"min"].sort()
        writer.writerow(
            [p["name"]
            ,standorte_alle[p["index_fastest"]]["name"]
            ,standorte_alle[p["index_fastest"]]["bereitschaft"]
            ,  ", ".join([standorte_alle[p["index_fastest"]]["lon"],standorte_alle[p["index_fastest"]]["lat"]])
            ,p["fastest_time"]
            ,standorte_81[p["index_fastest81er"]]["name"]
            ,standorte_81[p["index_fastest81er"]]["bereitschaft"]
            ,  ", ".join([standorte_alle[p["index_fastest"]]["lon"],standorte_alle[p["index_fastest"]]["lat"]])
            ,p["fastest_time81er"]
            ,standorte_84[p["index_fastest84er"]]["name"]
            ,standorte_84[p["index_fastest84er"]]["bereitschaft"]
            ,  ", ".join([standorte_alle[p["index_fastest"]]["lon"],standorte_alle[p["index_fastest"]]["lat"]])
            ,p["fastest_time84er"]
            ,  ", ".join(p["names_in"+str(delta_time)+"min"])]
        )
        # myfile.write("{0};{1};{2};{2},{1};{3};{4};{5};{6};{7};{8};{9};\n".format(p["name"],p["lon"],p["lat"],standorte_81[p["index_fastest81er"]]["name"],p["fastest_time81er"],standorte_81[p["index_shortest81er"]]["name"],p["shortest_dist81er"],in2min,time,dist))

print("fin_mix_gesamt")

fin_mix_gesamt


In [102]:
titlerow = ["Rettungspunkt", "Longitude", "Latitude", "Koordinaten", "Schnellestes Fahrzeug",
            "Anfahrtszeit", "Nächstes Fahrzeug", "Anfahrtsdistanz", "Ankunft innerhalb "+str(delta_time)+"min"]

all_times_title = []
all_dist_title = []
o81er_times_title = []
o81er_dist_title = []

for w in standorte_alle:
    all_times_title.append("Zeit_"+w["name"])
    all_dist_title.append("Entf_"+w["name"])
    if (w in standorte_81):
        o81er_times_title.append("Zeit_"+w["name"])
        o81er_dist_title.append("Entf_"+w["name"])
all_titlerow = copy.deepcopy(titlerow)
all_titlerow.extend(all_times_title)
all_titlerow.extend(all_dist_title)


with open("Orte_Bergwacht_gesamt.csv", 'w', encoding="utf-8",newline='') as myfile:
    writer = csv.writer(myfile,delimiter=",",quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(all_titlerow)

    for p in result:
        p["names_in"+str(delta_time)+"min"].sort()
        writer.writerow(
            [p["name"]
            ,p["lon"]
            ,p["lat"]
            ,p["lon"]+","+p["lat"]
            ,standorte_alle[p["index_fastest"]]["name"]
            ,p["fastest_time"]
            ,standorte_alle[p["index_shortest"]]["name"]
            ,p["shortest_dist"]
                ,  ", ".join(p["names_in"+str(delta_time)+"min"])]
            +p["formated_durations"]
            +p["formated_distances"])
print("fin_full_gesamt")
# exit()


# nur 81er  
with open("Orte_Bergwacht_nur81er_gesamt.csv", 'w', encoding="utf-8", newline='') as myfile:
    writer = csv.writer(myfile,delimiter=",",quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(titlerow)

    for p in result:
        p["names_in"+str(delta_time)+"min81er"].sort()
        writer.writerow(
            [p["name"]
            ,p["lon"]
            ,p["lat"]
            ,p["lon"]+","+p["lat"]
            ,standorte_81[p["index_fastest81er"]]["name"]
            ,p["fastest_time81er"]
            ,standorte_81[p["index_shortest81er"]]["name"]
            ,p["shortest_dist81er"]
                , ", ".join(p["names_in"+str(delta_time)+"min81er"])]
            +p["formated_durations81er"]
            +p["formated_distances81er"])
        # myfile.write("{0};{1};{2};{2},{1};{3};{4};{5};{6};{7};{8};{9};\n".format(p["name"],p["lon"],p["lat"],standorte_81[p["index_fastest81er"]]["name"],p["fastest_time81er"],standorte_81[p["index_shortest81er"]]["name"],p["shortest_dist81er"],in2min,time,dist))

print("fin_81er_gesamt")



# Überschneidung 2min
with open("Orte_Bergwacht_Überschneidungen_gesamt.csv", 'w', encoding="utf-8",newline='') as myfile:
    # myfile.write(string+time_str+dist_str+'\n')
    # for p in [x for x in result if len(x["index_in"+str(delta_time)+"min"])>1]:
    #     p["names_in"+str(delta_time)+"min"].sort()
    #     in"+str(delta_time)+"min = ", ".join(p["names_in"+str(delta_time)+"min"])
    #     time = "; ".join(p["formated_durations"])
    #     dist = "; ".join(p["formated_distances"])
    #     myfile.write("{0};{1};{2};{2},{1};{3};{4};{5};{6};{7};{8};{9};\n".format(p["name"],p["lon"],p["lat"],standorte_alle[p["index_fastest"]]["name"],p["fastest_time"],standorte_alle[p["index_shortest"]]["name"],p["shortest_dist"],in2min,time,dist))

    writer = csv.writer(myfile,delimiter=",",quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(all_titlerow)

    for p in [x for x in result if len(x["index_in"+str(delta_time)+"min"]) > 1]:
        p["names_in"+str(delta_time)+"min"].sort()
        writer.writerow(
            [p["name"]
            ,p["lon"]
            ,p["lat"]
            ,p["lon"]+","+p["lat"]
            ,standorte_alle[p["index_fastest"]]["name"]
            ,p["fastest_time"]
            ,standorte_alle[p["index_shortest"]]["name"]

            ,p["shortest_dist"]
                , ", ".join(p["names_in"+str(delta_time)+"min"])]
            +p["formated_durations"]
            +p["formated_distances"])
print("fin_full_overlap_"+str(i))


fin_full_gesamt
fin_81er_gesamt
fin_mix_gesamt
fin_full_overlap_1


# nur 81er

In [103]:
string="Rettungspunkt;Longitude;Latitude;Koordinaten;Schnellestes Fahrzeug;Anfahrtszeit;Nächstes Fahrzeug;Anfahrtsdistanz"
time_str=""
dist_str=""
for w in standorte_alle:
    time_str += ";Zeit "+w["name"]
    dist_str += ";Entfernung "+w["name"]
with open("Orte_Bergwacht_nur81er.csv", 'w', encoding="utf-8") as myfile:
    myfile.write(string+time_str+dist_str+';\n')
    
    for p in result:
        fastest = standorte_alle[p["index_fastest"]]["name"]
        print(fastest)
        shortest = standorte_alle[p["index_shortest"]]["name"]
        time = ("{0:.2f};{1:.2f};{2:.2f};{3:.2f};{4:.2f};{5:.2f}").format(*[x/60 for x in p["offset_durations"]]).replace('.',',')
        dist = ("{0:.3f};{1:.3f};{2:.3f};{3:.3f};{4:.3f};{5:.3f}").format(*p["distances"]).replace('.',',')
        myfile.write("{0};{1};{2};{2},{1};{3};{4};{5};{6};{7};{8};\n".format(p["name"],p["lon"],p["lat"],fastest,p["fastest_time81er"].replace('.',','),shortest,p["shortest_dist81er"].replace('.',','),time,dist))
print("fin")

Gr 81/1
Schöf 81/1
Schöf 81/1
Schöf 81/1
Schöf 81/1
Schöf 81/1
Schöf 81/1
Schöf 81/1
Schöf 81/1
Schöf 81/1
Schöf 81/1
Schöf 81/1
Schöf 81/1
Schöf 81/1
Gr 81/1
Gr 81/1
Gr 81/1
Schöf 81/1
Schöf 81/1
Gr 81/1
Gr 81/1
Mifi 84/1
Gr 81/1
Mifi 84/1
Mifi 84/1
Gr 81/1
Mifi 84/1
Mifi 84/1
HM 81/1
Mifi 84/1
Mifi 84/1
Mifi 84/1
Mifi 84/1
Mifi 84/1
Gr 81/1
Gr 81/1
HM 81/1
Gr 81/1
Mifi 84/1
HM 81/1
HM 81/1
HM 81/1
Gr 81/1
Mifi 84/1
HM 81/1
Mifi 84/1
Mifi 84/1
HM 81/1
Mifi 84/1
Mifi 84/1
Gr 81/1
Mifi 84/1
HM 81/1
Gr 81/1
Gr 81/1
Gr 81/1
WK 84/1
WK 84/1
Gr 81/1
Gr 81/1
WK 84/1
WK 84/1
Gr 81/1
Gr 81/1
Gr 81/1
WK 84/1
Gr 81/1
Gr 81/1
Gr 81/1
Gr 81/1
Gr 81/1
Gr 81/1
Gr 81/1
Gr 81/1
Gr 81/1
Gr 81/1
Gr 81/1
Gr 81/1
Gr 81/1
Gr 81/1
Gr 81/1
Gr 81/1
Gr 81/1
Gr 81/1
Gr 81/1
Gr 81/1
Gr 81/1
Gr 81/1
Gr 81/1
Gr 81/1
Gr 81/1
Gr 81/1
Gr 81/1
Gr 81/1
Gr 81/1
Gr 81/1
Gr 81/1
Gr 81/1
Gr 81/1
Gr 81/1
Gr 81/1
Gr 81/1
Gr 81/1
Gr 81/1
Gr 81/1
Gr 81/1
Gr 81/1
Gr 81/1
Gr 81/1
Gr 81/1
Gr 81/1
Gr 81/1
Gr 81/1
Gr 81/1
Gr 81/1
G